<a href="https://colab.research.google.com/github/JHhyuk/Case-Study_Fake-Product-Review-Monitoring-and-Removal/blob/master/review_final4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install wordcloud
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 5.8 MB 29.6 MB/s 
     |████████████████████████████████| 182 kB 78.7 MB/s 
     |████████████████████████████████| 7.6 MB 62.4 MB/s 


In [2]:
import torch
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import torch.nn as nn
import torch.optim as optim

from transformers import BertTokenizer, BertForSequenceClassification

from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import seaborn as sns

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
print(device)

cuda


In [5]:
path = '/content/drive/MyDrive/amazon_review.csv'

df = pd.read_csv(path,encoding='cp949')
print(df.shape)
df.head()

(1597, 4)


,categories,name,rating,_text
0,"Amazon Devices,mazon.co.uk",Kindle Paperwhite,5.0,I initially had trouble deciding between the p...
1,"Amazon Devices,mazon.co.uk",Kindle Paperwhite,5.0,Allow me to preface this with a little history...
2,"Amazon Devices,mazon.co.uk",Kindle Paperwhite,4.0,I am enjoying it so far. Great for reading. Ha...
3,"Amazon Devices,mazon.co.uk",Kindle Paperwhite,5.0,I bought one of the first Paperwhites and have...
4,"Amazon Devices,mazon.co.uk",Kindle Paperwhite,5.0,I have to say upfront - I don't like coroporat...


In [6]:
df = df.rename(columns={'_text':'text_'})
df['label'] = ['OR']*len(df)
df.head()

,categories,name,rating,text_,label
0,"Amazon Devices,mazon.co.uk",Kindle Paperwhite,5.0,I initially had trouble deciding between the p...,OR
1,"Amazon Devices,mazon.co.uk",Kindle Paperwhite,5.0,Allow me to preface this with a little history...,OR
2,"Amazon Devices,mazon.co.uk",Kindle Paperwhite,4.0,I am enjoying it so far. Great for reading. Ha...,OR
3,"Amazon Devices,mazon.co.uk",Kindle Paperwhite,5.0,I bought one of the first Paperwhites and have...,OR
4,"Amazon Devices,mazon.co.uk",Kindle Paperwhite,5.0,I have to say upfront - I don't like coroporat...,OR


In [7]:
tokenizer = BertTokenizer.from_pretrained('bert-base-cased')

Downloading:   0%|          | 0.00/213k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [8]:
labels = {'CG':0, # 가짜면 0
          'OR':1 # 진짜면 1
          }

class Dataset(torch.utils.data.Dataset):

    def __init__(self, df):

        self.labels = [labels[label] for label in df['label']] # df 컬럼 잘 보고 설정
        self.texts = [tokenizer(text, 
                               padding='max_length', max_length = 128, truncation=True,
                                return_tensors="pt") for text in df['text_']] # df 컬럼 잘 보고 설정

    def classes(self):
        return self.labels

    def __len__(self):
        return len(self.labels)

    def get_batch_labels(self, idx):
        # Fetch a batch of labels
        return np.array(self.labels[idx])

    def get_batch_texts(self, idx):
        # Fetch a batch of inputs
        return self.texts[idx]

    def __getitem__(self, idx):

        batch_texts = self.get_batch_texts(idx)
        batch_y = self.get_batch_labels(idx)

        return batch_texts, batch_y

In [9]:
from torch import nn
from transformers import BertModel

class BertClassifier(nn.Module):

    def __init__(self, dropout=0.5):

        super(BertClassifier, self).__init__()

        self.bert = BertModel.from_pretrained('bert-base-cased') # 'bert-base-cased': 영어용, 다른 언어의 데이터면 다른거 쓰기
        self.dropout = nn.Dropout(dropout)
        self.linear = nn.Linear(768, 2)
        self.relu = nn.ReLU()

    def forward(self, input_id, mask):

        _, pooled_output = self.bert(input_ids= input_id, attention_mask=mask,return_dict=False)
        # _: embedding vectors of all of the tokens in a sequence
        # pooled_output: embedding vector of [CLS] token, 분류 문제에선 이것만 써도 충분
        dropout_output = self.dropout(pooled_output)
        linear_output = self.linear(dropout_output)
        final_layer = self.relu(linear_output)

        return final_layer

In [10]:
model = BertClassifier()

Downloading:   0%|          | 0.00/436M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [11]:
result=[]
def evaluate(model, data):

    test = Dataset(data)

    dataloader = torch.utils.data.DataLoader(test, batch_size=1)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    if use_cuda:

        model = model.cuda()
    path = "/content/drive/MyDrive/ClassifierModel.pth"
    model.load_state_dict(torch.load(path))
    total_acc_test = 0
    with torch.no_grad():

        for input, label in dataloader:

              label = label.to(device)
              mask = input['attention_mask'].to(device)
              input_id = input['input_ids'].squeeze(1).to(device)

              output = model(input_id, mask)
              if(output.argmax(dim=1)[0]==1):
                result.append("OR")
              else:
                result.append("CG")
              
              acc = (output.argmax(dim=1) == label).sum().item()
              total_acc_test += acc
    
    print(result)
    print(len(data))
    print(total_acc_test)
    print(f'Test Accuracy: {total_acc_test / len(data): .3f}')
    
 
evaluate(model, df)


['OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR', 'OR

In [12]:
df['predicted_label'] = result
print(df.shape)
df.head()

(1597, 6)


,categories,name,rating,text_,label,predicted_label
0,"Amazon Devices,mazon.co.uk",Kindle Paperwhite,5.0,I initially had trouble deciding between the p...,OR,OR
1,"Amazon Devices,mazon.co.uk",Kindle Paperwhite,5.0,Allow me to preface this with a little history...,OR,OR
2,"Amazon Devices,mazon.co.uk",Kindle Paperwhite,4.0,I am enjoying it so far. Great for reading. Ha...,OR,OR
3,"Amazon Devices,mazon.co.uk",Kindle Paperwhite,5.0,I bought one of the first Paperwhites and have...,OR,OR
4,"Amazon Devices,mazon.co.uk",Kindle Paperwhite,5.0,I have to say upfront - I don't like coroporat...,OR,OR


In [13]:
df2 = df.loc[(df['predicted_label'] == 'OR') & (df['rating'] != 3) ]
print(df2.shape)

(1443, 6)


In [14]:
df2['division'] = np.select([df2.rating > 3], [1], default=0)
positive = len(df2.loc[df2['rating'] > 3])
negative = len(df2.loc[df2['rating'] < 3])
print(f'Positive review : ', positive)
print(f'Negative review : ', negative)
df2[0:5]


Positive review :  951
Negative review :  73


<ipython-input-14-970e372bb7b5>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2['division'] = np.select([df2.rating > 3], [1], default=0)


,categories,name,rating,text_,label,predicted_label,division
0,"Amazon Devices,mazon.co.uk",Kindle Paperwhite,5.0,I initially had trouble deciding between the p...,OR,OR,1
1,"Amazon Devices,mazon.co.uk",Kindle Paperwhite,5.0,Allow me to preface this with a little history...,OR,OR,1
2,"Amazon Devices,mazon.co.uk",Kindle Paperwhite,4.0,I am enjoying it so far. Great for reading. Ha...,OR,OR,1
3,"Amazon Devices,mazon.co.uk",Kindle Paperwhite,5.0,I bought one of the first Paperwhites and have...,OR,OR,1
4,"Amazon Devices,mazon.co.uk",Kindle Paperwhite,5.0,I have to say upfront - I don't like coroporat...,OR,OR,1


In [29]:
df3 = df2.loc[df2['categories']=='Amazon Devices,Kindle Store']
print(df3.shape)
# df3 = df3.drop(['rating', 'label', 'division'], axis=1)
df3.head()

(139, 8)


,categories,name,rating,text_,label,predicted_label,division,predicted_division
473,"Amazon Devices,Kindle Store",All-New Amazon Fire 7 Tablet Case (7th Generation,5.0,"Over the years, I've tried a variety of cases ...",OR,OR,1,1
474,"Amazon Devices,Kindle Store",All-New Amazon Fire 7 Tablet Case (7th Generation,2.0,I typically don't post reviews however don't w...,OR,OR,0,0
475,"Amazon Devices,Kindle Store",All-New Amazon Fire 7 Tablet Case (7th Generation,5.0,All good A+,OR,OR,1,1
476,"Amazon Devices,Kindle Store",All-New Amazon Fire 7 Tablet Case (7th Generation,4.0,This case is very well made and sturdy. It per...,OR,OR,1,1
477,"Amazon Devices,Kindle Store",All-New Amazon Fire 7 Tablet Case (7th Generation,4.0,It's pretty and matches my Fire tablet and I l...,OR,OR,1,1


In [30]:
class Dataset(torch.utils.data.Dataset):

    def __init__(self, df3):

        self.divisions = [division for division in df3['division']]
        self.texts = [tokenizer(text, 
                               padding='max_length', max_length = 128, truncation=True,
                                return_tensors="pt") for text in df3['text_']]
    def classes(self):
        return self.divisions

    def __len__(self):
        return len(self.divisions)

    def get_batch_divisions(self, idx):
        # Fetch a batch of labels
        return np.array(self.divisions[idx])

    def get_batch_texts(self, idx):
        # Fetch a batch of inputs
        return self.texts[idx]

    def __getitem__(self, idx):

        batch_texts = self.get_batch_texts(idx)
        batch_y = self.get_batch_divisions(idx)

        return batch_texts, batch_y


In [31]:
result2=[]
def evaluate(model, test_data):

    test = Dataset(test_data)

    test_dataloader = torch.utils.data.DataLoader(test, batch_size=1)

    use_cuda = torch.cuda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")

    if use_cuda:

        model = model.cuda()
    checking_num1 = 0
    checking_num2 = 0
    checking_num3 = 0
    checking_num4 = 0
    path = "/content/drive/MyDrive/NetModel_s.pth"
    model.load_state_dict(torch.load(path))    
    total_acc_test = 0
    with torch.no_grad():

        for test_input, test_division in test_dataloader:

              test_division = test_division.to(device)
              mask = test_input['attention_mask'].to(device)
              input_id = test_input['input_ids'].squeeze(1).to(device)

              output = model(input_id, mask)
              if output.argmax(dim=1)[0] == 1:
                  result2.append(1)
              else:
                  result2.append(0)

              acc = (output.argmax(dim=1) == test_division).sum().item()
              total_acc_test += acc
    
    print(f'Test Accuracy: {total_acc_test / len(test_data): .3f}')
    print('==',total_acc_test)
    print('==',len(test_data))

evaluate(model, df3)

Test Accuracy:  0.806
== 112
== 139


In [32]:
df3['predicted_division'] = result2
print(df3.shape)
df3.head() 

(139, 8)


<ipython-input-32-5bcc8440df72>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df3['predicted_division'] = result2


,categories,name,rating,text_,label,predicted_label,division,predicted_division
473,"Amazon Devices,Kindle Store",All-New Amazon Fire 7 Tablet Case (7th Generation,5.0,"Over the years, I've tried a variety of cases ...",OR,OR,1,1
474,"Amazon Devices,Kindle Store",All-New Amazon Fire 7 Tablet Case (7th Generation,2.0,I typically don't post reviews however don't w...,OR,OR,0,0
475,"Amazon Devices,Kindle Store",All-New Amazon Fire 7 Tablet Case (7th Generation,5.0,All good A+,OR,OR,1,1
476,"Amazon Devices,Kindle Store",All-New Amazon Fire 7 Tablet Case (7th Generation,4.0,This case is very well made and sturdy. It per...,OR,OR,1,1
477,"Amazon Devices,Kindle Store",All-New Amazon Fire 7 Tablet Case (7th Generation,4.0,It's pretty and matches my Fire tablet and I l...,OR,OR,1,1


In [33]:
positive = len(df3.loc[df3['predicted_division'] == 1])
negative = len(df3.loc[df3['predicted_division'] == 0])
print(f'Positive Review : ', positive)
print(f'Negative Review : ', negative)
print(f'Positive Review rate :  {(positive/(positive+negative))*100: .3f}')

Positive Review :  71
Negative Review :  68
Positive Review rate :   51.079


In [34]:
df3 = df3.drop(['rating', 'label', 'division'], axis=1)
df3.head()
  

,categories,name,text_,predicted_label,predicted_division
473,"Amazon Devices,Kindle Store",All-New Amazon Fire 7 Tablet Case (7th Generation,"Over the years, I've tried a variety of cases ...",OR,1
474,"Amazon Devices,Kindle Store",All-New Amazon Fire 7 Tablet Case (7th Generation,I typically don't post reviews however don't w...,OR,0
475,"Amazon Devices,Kindle Store",All-New Amazon Fire 7 Tablet Case (7th Generation,All good A+,OR,1
476,"Amazon Devices,Kindle Store",All-New Amazon Fire 7 Tablet Case (7th Generation,This case is very well made and sturdy. It per...,OR,1
477,"Amazon Devices,Kindle Store",All-New Amazon Fire 7 Tablet Case (7th Generation,It's pretty and matches my Fire tablet and I l...,OR,1


In [37]:
or1 = len(df.loc[(df['name'] == 'All-New Amazon Fire 7 Tablet Case (7th Generation') & (df['predicted_label']=='OR')])
cg1 = len(df.loc[(df['name'] == 'All-New Amazon Fire 7 Tablet Case (7th Generation') & (df['predicted_label']=='CG')])
or2 = len(df.loc[(df['name'] == 'All-New Amazon Fire HD 8 Tablet Case (7th Generation') & (df['predicted_label']=='OR')])
cg2 = len(df.loc[(df['name'] == 'All-New Amazon Fire HD 8 Tablet Case (7th Generation') & (df['predicted_label']=='CG')])
print(f'All-New Amazon Fire 7 Tablet Case (7th Generation)의 리뷰 신뢰성 :  {(or1/(or1+cg1))*100: .3f}%')
print(f'All-New Amazon Fire HD 8 Tablet Case (7th Generation)의 리뷰 신뢰성 :  {(or2/(or2+cg2))*100: .3f}%')

positive1 = len(df3.loc[(df3['name'] == 'All-New Amazon Fire 7 Tablet Case (7th Generation') & (df3['predicted_division']==1)])
negative1 = len(df3.loc[(df3['name'] == 'All-New Amazon Fire 7 Tablet Case (7th Generation') & (df3['predicted_division']==0)])
positive2 = len(df3.loc[(df3['name'] == 'All-New Amazon Fire HD 8 Tablet Case (7th Generation') & (df3['predicted_division']==1)])
negative2 = len(df3.loc[(df3['name'] == 'All-New Amazon Fire HD 8 Tablet Case (7th Generation') & (df3['predicted_division']==0)])
print(f'')
print(f'All-New Amazon Fire 7 Tablet Case (7th Generation)의 선호도 :  {(positive1/(positive1+negative1))*100: .3f}%')
print(f'All-New Amazon Fire HD 8 Tablet Case (7th Generation)의 선호도 :  {(positive2/(positive2+negative2))*100: .3f}%')
print(f'All-New Amazon Fire 7 Tablet Case (7th Generation)을 추천')

All-New Amazon Fire 7 Tablet Case (7th Generation)의 리뷰 신뢰성 :   96.296%
All-New Amazon Fire HD 8 Tablet Case (7th Generation)의 리뷰 신뢰성 :   94.444%

All-New Amazon Fire 7 Tablet Case (7th Generation)의 선호도 :   70.833%
All-New Amazon Fire HD 8 Tablet Case (7th Generation)의 선호도 :   61.538%
All-New Amazon Fire 7 Tablet Case (7th Generation)을 추천
